<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#TF-Estimator-MNIST-Example" data-toc-modified-id="TF-Estimator-MNIST-Example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TF Estimator MNIST Example</a></span></li><li><span><a href="#TF-Slim-MNIST-Example" data-toc-modified-id="TF-Slim-MNIST-Example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>TF Slim MNIST Example</a></span></li><li><span><a href="#TFLearn-MNIST-Example" data-toc-modified-id="TFLearn-MNIST-Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TFLearn MNIST Example</a></span></li><li><span><a href="#Pretty-Tensor-MNIST-Example" data-toc-modified-id="Pretty-Tensor-MNIST-Example-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pretty Tensor MNIST Example</a></span></li><li><span><a href="#Sonnet-MNIST-Example" data-toc-modified-id="Sonnet-MNIST-Example-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Sonnet MNIST Example</a></span></li></ul></div>

# TF Estimator MNIST Example 

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=False
                                  )
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
n_classes = 10
batch_size = 100
n_steps = 1000
learning_rate = 0.01


In [4]:
tf.reset_default_graph()


In [5]:
def model_fn(features, labels, mode):
    """ define the model function
    """
    espec_op = tf.estimator.EstimatorSpec
    # features is a dict as per Estimator specifications
    x = features['images']
    # define the network
    layer_1 = tf.layers.dense(x, 32)
    layer_2 = tf.layers.dense(layer_1, 32)
    logits = tf.layers.dense(layer_2, n_classes)

    # define predicted classes
    predicted_classes = tf.argmax(logits, axis=1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        espec = espec_op(mode,
                         predictions=predicted_classes
                         )
    else:
        # define loss and optimizer
        entropy_op = tf.nn.sparse_softmax_cross_entropy_with_logits
        loss_op = tf.reduce_mean(entropy_op(logits=logits,
                                            labels=tf.cast(labels,
                                                           dtype=tf.int32)
                                            )
                                 )
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss_op, global_step=tf.train.get_global_step())

        # define accuracy
        accuracy_op = tf.metrics.accuracy(
            labels=labels, predictions=predicted_classes)

        espec = espec_op(mode=mode,
                         predictions=predicted_classes,
                         loss=loss_op,
                         train_op=train_op,
                         eval_metric_ops={'accuracy': accuracy_op}
                         )

    return espec




In [6]:
# create estimator object
model = tf.estimator.Estimator(model_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpzpp7cwo3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09eaf5f390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)
model.train(train_input_fn, steps=n_steps)


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpzpp7cwo3/model.ckpt.
INFO:tensorflow:loss = 2.3743649, step = 0
INFO:tensorflow:global_step/sec: 472.691
INFO:tensorflow:loss = 1.334875, step = 100 (0.214 sec)
INFO:tensorflow:global_step/sec: 524.455
INFO:tensorflow:loss = 1.0205415, step = 200 (0.191 sec)
INFO:tensorflow:global_step/sec: 530.567
INFO:tensorflow:loss = 0.7639195, step = 300 (0.187 sec)
INFO:tensorflow:global_step/sec: 525.177
INFO:tensorflow:loss = 0.63917464, step = 400 (0.190 sec)
INFO

In [8]:
# evaluate the model
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_test},
    y=y_test,
    batch_size=batch_size,
    shuffle=False)
model.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-30-04:51:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzpp7cwo3/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-30-04:51:43
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.8877, global_step = 1000, loss = 0.40761253
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpzpp7cwo3/model.ckpt-1000


{'accuracy': 0.8877, 'loss': 0.40761253, 'global_step': 1000}